In [58]:
import pandas as pd

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold

In [60]:
df = pd.read_csv('2013-2018년_가스공급량과_기온2.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,연월일,시간,구분,공급량,year,month,day,weekday,기온
0,0,0,2013-01-01,1,0,2497.129,2013,1,1,1,-6.4
1,1,1,2013-01-01,1,1,2169.093,2013,1,1,1,-6.4
2,2,2,2013-01-01,1,2,226.178,2013,1,1,1,-6.4
3,3,3,2013-01-01,1,3,1434.516,2013,1,1,1,-6.4
4,4,4,2013-01-01,1,4,3272.837,2013,1,1,1,-6.4


In [61]:
# col 선택 : year, month, day, 시간, 기온
# 타겟 : 공급량

# 데이터 확인 

In [62]:
col = ['year', 'month', 'day', '시간', '기온', '공급량']
df_col = df[col]
df_col.head()

,year,month,day,시간,기온,공급량
0,2013,1,1,1,-6.4,2497.129
1,2013,1,1,1,-6.4,2169.093
2,2013,1,1,1,-6.4,226.178
3,2013,1,1,1,-6.4,1434.516
4,2013,1,1,1,-6.4,3272.837


# 모델 평가

In [63]:
col = ['year', 'month', 'day', '시간']
X = df[col]
y = df['공급량']

### 데이터 분할

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((276066, 4), (92022, 4), (276066,), (92022,))

### 선형회귀 모델 구축

In [65]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### 교차 검증

In [66]:
kfold = KFold(shuffle=True)
scores = cross_val_score(model, X_test, y_test, cv=kfold, scoring='r2')

print(f"훈련 결정계수 = {model.score(X_train, y_train)}")
print(f"테스트 결정계수 = {model.score(X_test, y_test)}")
print(f"결정계수 평균 = {scores.mean()}")

훈련 결정계수 = 0.033742996403450576
테스트 결정계수 = 0.03444233489950621
결정계수 평균 = 0.034249326631524045


### Lasso 모델 구축 및 평가

In [67]:
from sklearn.linear_model import Lasso

In [68]:
lasso = Lasso(alpha=0.01, random_state=0)
lasso.fit(X_train, y_train)

kfold = KFold(shuffle=True)
scores = cross_val_score(lasso, X_test, y_test, cv=kfold, scoring='r2')

print(f"훈련 결정계수 = {lasso.score(X_train, y_train)}")
print(f"테스트 결정계수 = {lasso.score(X_test, y_test)}")
print(f"결정계수 평균 = {scores.mean()}")

훈련 결정계수 = 0.033742996349721666
테스트 결정계수 = 0.03444232579216555
결정계수 평균 = 0.03428574024999849


### Ridge 모델 구축 및 평가

In [69]:
from sklearn.linear_model import Ridge

In [70]:
ridge = Lasso(alpha=0.0001, random_state=0)
ridge.fit(X_train, y_train)

kfold = KFold(shuffle=True)
scores = cross_val_score(ridge, X_test, y_test, cv=kfold, scoring='r2')

print(f"훈련 결정계수 = {ridge.score(X_train, y_train)}")
print(f"테스트 결정계수 = {ridge.score(X_test, y_test)}")
print(f"결정계수 평균 = {scores.mean()}")

훈련 결정계수 = 0.03374299640344536
테스트 결정계수 = 0.034442334807526565
결정계수 평균 = 0.03434514588277713


# 기온 데이터 포함 모델 평가

### 데이터 정규화, 표준화

In [71]:
### 정규화, 표준화
col = ['year', 'month', 'day', '시간', '기온']
X = df[col]
y = df['공급량']

# 정규화
mmscaler = MinMaxScaler()
m_X = mmscaler.fit_transform(X)
# 표준화
sscaler = StandardScaler()
s_m_X = sscaler.fit_transform(m_X)

### 데이터 분할

In [72]:
X_train, X_test, y_train, y_test = train_test_split(s_m_X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((276066, 5), (92022, 5), (276066,), (92022,))

### 선형회귀 모델 구축

In [73]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### 교차 검증

In [74]:
kfold = KFold(shuffle=True)
scores = cross_val_score(model, X_test, y_test, cv=kfold, scoring='neg_mean_squared_error')

print(f"훈련 결정계수 = {model.score(X_train, y_train)}")
print(f"테스트 결정계수 = {model.score(X_test, y_test)}")
print(f"MSE = {-scores.mean()}")

훈련 결정계수 = 0.2657516396162296
테스트 결정계수 = 0.26371252420825086
MSE = 634951.6163939991


### Lasso 모델 구축 및 평가

In [75]:
lasso = Lasso(alpha=0.01, random_state=0)
lasso.fit(X_train, y_train)

kfold = KFold(shuffle=True)
scores = cross_val_score(lasso, X_test, y_test, cv=kfold, scoring='r2')

print(f"훈련 결정계수 = {lasso.score(X_train, y_train)}")
print(f"테스트 결정계수 = {lasso.score(X_test, y_test)}")
print(f"결정계수 평균 = {scores.mean()}")

훈련 결정계수 = 0.2657516390435516
테스트 결정계수 = 0.26371253949680473
결정계수 평균 = 0.2635989232314893


### Ridge 모델 구축 및 평가

In [76]:
ridge = Lasso(alpha=0.0001, random_state=0)
ridge.fit(X_train, y_train)

kfold = KFold(shuffle=True)
scores = cross_val_score(ridge, X_test, y_test, cv=kfold, scoring='r2')

print(f"훈련 결정계수 = {ridge.score(X_train, y_train)}")
print(f"테스트 결정계수 = {ridge.score(X_test, y_test)}")
print(f"결정계수 평균 = {scores.mean()}")

훈련 결정계수 = 0.2657516396161723
테스트 결정계수 = 0.26371252435989034
결정계수 평균 = 0.26357272502658236
